In [54]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import datetime
import pandas

In [ ]:
ref_ids = """92010366
92011384
92012423
92025294
92025294
92020900
92002347
92009897
92009897
92013184
25023108
92025060
92025062"""
ref_ids = ref_ids.split("\n")

In [ ]:
def get_html(url):
    global actual_proxy
    while True:
#         html = requests.get(url, proxies = get_proxy(), timeout=5)
        html = requests.get(url)
        if html.status_code == 200:
            return html
        elif html.status_code == 542:
            time.sleep(2)
        else:
            return "-"

In [72]:
def get_subitems(html):
    sitems = html.find("table", class_="list-multiproduit")
    if not sitems:
        return []
    sitems = sitems.find_all("tr")
    return sitems
    
def get_libelle(html):
    lib = html.find("div", class_="product-name")
    if not lib:
        return "-"
    lib = lib.h1.span.text.strip()
    return lib

def get_sublibelle(item, mode):
    if not mode:
        return []
    lib1 = item.find_all("td")
    lib1 = [x.text for x in lib1 if not x.has_attr('class')]
    return lib1

def get_main_prix(item):
    price = item.find("span", class_="price")
    if not price:
        return "-"
    else:
        return price.text.strip()

def get_prix_per_unit(item):
    price2 = item.find("span", class_="price-total")
    if price2 and "soit" in price2.text and "/" in price2.text:
        return unidecode(price2.text.split("soit")[1])
    price2 = item.find("span", class_="price")
    if price2 and "/" in price2.text:
        return unidecode(price2.text.split("TTC")[0].strip())
    return "-"

def get_prix_unit(item):
    price2 = item.find("span", class_="price-total")
    if price2 and "soit" in price2.text and "/" in price2.text:
        return price2.text.split("/")[1]
    price2 = item.find("span", class_="price")
    if price2 and "/" in price2.text:
        return price2.text.split("/")[1]
    return "-"

def get_photo(item, html_mode, libelle):
    if not html_mode:
        image = item.findAll("img", {"itemprop":"image"})
        return image[0]['src']
    else:
        image = item.find("img", {"alt":libelle})
        if not image:
            return "-"
        if "http" in image['src']:
            return image['src']
        return image['data-frz-src']


In [19]:
def scrap_products(url, data):
    sample = get_html(url)
    html = BeautifulSoup(sample.content.decode('utf-8', 'ignore'))
    items = get_items(html)
    url_item = get_url(items[0])
    obtain_info(url_item, data)

In [65]:
def obtain_info(url, data, ref):
    sample = get_html(url + "?frz-smartcache-fragment=true&frz-timeout=5000&frz-smartcache-v=2&frz-smartcache-placeholders-number=3")
    html = sample.text.replace("\\n", "").replace("\\","").replace("            ", " ")
    html = html[html.find("\"fstrz-scss-2\":{\"content\":") + 27:]
    html = BeautifulSoup(html)
    libelle = get_libelle(html).strip()
    photo = get_photo(html, True, libelle)
    items = get_subitems(html)
    html_mode = False
    if not items:
        html_mode = True
        items = [html]
    for item in items:
        data["Id Magasin"].append(shop_id)
        data["Enseigne"].append(shop_city)
        data["Code Produit"].append(ref)
        data["Libelle"].append(" ".join([libelle] + get_sublibelle(item, html_mode)))
        data["Prix"].append(get_main_prix(item))
        data["Prix Unitaire"].append(get_prix_per_unit(item))
        data["Unite Px Unitaire"].append(get_prix_unit(item))
        data["Prix Num"].append(".".join(data["Prix"][-1].split("€")))
        data["DateRecup"].append(datetime.datetime.today().strftime("%Y/%m/%d"))
        data["HeureRecup"].append(datetime.datetime.today().strftime("%H:%M:%S.%s"))
        data["Lien Photo"].append(photo)
        data["Lien"].append(url)

In [ ]:
shop_id = "miramas"
shop_city = "Miramas"
data = ini_data()
for ref in ref_ids:
    url = f"http://www.bricocash.fr/{shop_id}/catalogsearch/result/?q={ref}"
    print(url)
    scrap_products(url, data)
pandas.DataFrame(data).to_csv(f"output_bricocash.csv", sep=';', index=False)
